In [62]:
import pandas as pd
import geopandas as gpd
import numpy as np
import re

In [63]:
# read in shapefile file
all_cases = gpd.read_file(r'C:\Users\kevan\OneDrive\Desktop\Data+\DevCasesSHP\Development_Cases.shp')

In [64]:
# types = A_TYPE codes we want to keep
types = ['PL_MINSP', 'PL_SSP_SM', 'PL_SSM_SM2', 'PL_CPAA', 'PL_MINPP', 'PL_MAJSP', 'PL_MAJSUP', 'PL_PPA', 'PL_MAJPP'] 
# filter that only keeps cases of specified types
filter_cases_type = all_cases[all_cases['A_TYPE'].isin(types)]
# status = A_STATUS codes we want to keep
status = all_cases['A_STATUS'].unique()
status = status[~np.isin(status, ['WITH', 'VOID','DEN','DISAP','EXP'])]
# filter that only keeps cases of specified status
filter_cases_status = filter_cases_type[filter_cases_type['A_STATUS'].isin(status)]

In [65]:
keywords = ['home', 'family', 'residen', 'mixed', 'mized', 'duplex', 'apartment', ' housing', 'condo', 'dwelling', 'tenant', 'affordable', 'units', 'townhouse']
pattern = '|'.join(keywords)
filtered1 = filter_cases_status[filter_cases_status['A_DESCRIPT'].str.contains(pattern, case=False, na=False)]

#use not pattern to avoid these words
keywords_avoid = ['expand','storage']
pattern_avoid = '|'.join(keywords_avoid)
filtered2 = filtered1[~filtered1['A_DESCRIPT'].str.contains(pattern_avoid, case=False, na=False)]

In [66]:
#filter for all status dates after 2020
filtered2['A_STATUS_D'] = pd.to_datetime(filtered2['A_STATUS_D'])
filtered3 = filtered2[filtered2['A_STATUS_D'].dt.year>=2020]

c:\Users\kevan\anaconda3\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [67]:
#define extract_units function number 1

# def normalize_for_regex(term):
#     return re.sub(r'[-\s]+', r'\\s*-?\\s*', term)

# def extract_units1(description):
#     # Remove square footage
#     description = re.sub(
#         r'(\d+|\d{1,3}(,\d{3})*)(\s+[A-Za-z-]+){0,2}?\s*(SF|square feet|sq\.?\s*ft\.?|sqft)',
#         '', description, flags=re.IGNORECASE
#     )

#     # Housing normalization, THIS MEANS THAT IF HOUSES->HOME, S-F -> SINGLE FAMILY, ETC.
#     term_map = {
#         "home": "home", "homes": "home", "house": "home", "houses": "home",
#         "duplex": "duplex", "duplexes": "duplex",
#         "condo": "condo", "condominium": "condo", "condominiums": "condo", "condos": "condo", 
#         "apartment": "apartment", "apartments": "apartment",
#         "townhome": "townhouse", "townhomes": "townhouse",
#         "townhouse": "townhouse", "townhouses": "townhouse",
#         "town home": "townhouse", "town homes": "townhouse",
#         "town house": "townhouse", "town houses": "townhouse",
#         "multifamily": "multifamily", "multi-family": "multifamily", "multi - family": "multifamily", "multi family": "multifamily",
#         "MF": "multifamily", "mutifamily": "multifamily",
#         "single family": "single family", "single-family": "single family", 
#         "single - family": "single family", "s-f": "single family", "s - f": "single family", "s f": "single family"
#     }

#     # Optional leading and trailing terms
#     modifiers = ["attached", "detached"]
#     suffixes = ["units", "lots", "homes", "houses"]

#     # Build regex patterns
#     housing_pattern = "|".join([normalize_for_regex(term) for term in term_map])
#     modifier_pattern = "|".join(modifiers)
#     suffix_pattern = "|".join(suffixes)

#     match_pattern = rf'''
#         \b
#         (?P<qty>\d{{1,4}})
#         \s+
#         (?:(?P<mod>{modifier_pattern})\s+)?
#         (?:[A-Za-z-]+\s+){{0,2}}?
#         (?P<type>{housing_pattern})
#         (?:\s+(?P<suffix>{suffix_pattern}))?
#         \b
#     '''

#     matches = re.finditer(match_pattern, description, flags=re.IGNORECASE | re.VERBOSE)
    
#     result = []
#     for match in matches:
#         qty = match.group("qty")
#         raw_type = match.group("type")
#         raw_mod = match.group("mod")
#         raw_suffix = match.group("suffix")

#         # Normalize type
#         norm_key = re.sub(r'[-\s]+', ' ', raw_type.lower()).strip()
#         normalized_type = term_map.get(norm_key, norm_key)

#         # Build output tuple
#         result.append((
#             int(qty),
#             raw_mod.lower() if raw_mod else None,
#             normalized_type,
#             raw_suffix.lower() if raw_suffix else None
#         ))

#     return result

In [68]:
def normalize_for_regex(term):
    return re.sub(r'[-\s]+', r'\\s*-?\\s*', term)

def extract_units(description):
    # Remove square footage references
    description = re.sub(
        r'(\d+|\d{1,3}(,\d{3})*)(\s+[A-Za-z-]+){0,2}?\s*(SF|square feet|sq\.?\s*ft\.?|sqft)',
        '', description, flags=re.IGNORECASE
    )

    # Map variations to standardized types
    term_map = {
        "home": "home", "homes": "home", "house": "home", "houses": "home",
        "duplex": "duplex", "duplexes": "duplex",
        "condo": "condo", "condominium": "condo", "condominiums": "condo", "condos": "condo", 
        "apartment": "apartment", "apartments": "apartment",
        "townhome": "townhouse", "townhomes": "townhouse",
        "townhouse": "townhouse", "townhouses": "townhouse",
        "town home": "townhouse", "town homes": "townhouse",
        "town house": "townhouse", "town houses": "townhouse",
        "multifamily": "multifamily", "multi-family": "multifamily", 
        "multi - family": "multifamily", "multi family": "multifamily",
        "mutifamily": "multifamily", "MF": "multifamily",
        "single family": "single family", "single-family": "single family", 
        "single - family": "single family", "s-f": "single family", "s - f": "single family", "s f": "single family"
    }

    modifiers = ["attached", "detached"]
    suffixes = ["units", "lots", "homes", "houses"]

    housing_pattern = "|".join([normalize_for_regex(term) for term in term_map])
    modifier_pattern = "|".join(modifiers)
    suffix_pattern = "|".join(suffixes)

    # Extended match pattern to support both "qty before type" and "type before qty"
    match_pattern = rf'''
    (?:
        # Qty before type
        (?P<qty>\(?\d{{1,4}}\)?)
        (?:\s*[-+&/]?\s*)?
        (?:({modifier_pattern})\s*){{0,2}}?
        (?:\w+\s*){{0,6}}?
        (?P<type>{housing_pattern})
        (?:\s+({modifier_pattern}))?
        (?:\s+(?P<suffix>{suffix_pattern}))?

    |
        # Type before qty
        (?P<type2>{housing_pattern})
        (?:\s+({modifier_pattern}))?
        (?:\s*[-+&/]?\s*)?
        (?:\w+\s*){{0,6}}?
        (?P<qty2>\(?\d{{1,4}}\)?)
        (?:\s+(?P<suffix2>{suffix_pattern}))?

    |
        # Type with quantity in parentheses
        (?P<type3>{housing_pattern})
        (?:\s+\w+){{0,6}}?
        \(\s*(?P<qty3>\d{{1,4}})\s+(?P<suffix3>{suffix_pattern})\s*\)
    )
'''

    matches = re.finditer(match_pattern, description, flags=re.IGNORECASE | re.VERBOSE)
    
    result = []
    for match in matches:
        qty = match.group("qty") or match.group("qty2")
        raw_type = match.group("type") or match.group("type2")
        raw_mod = match.group(2)  # first modifier (position varies)
        raw_suffix = match.group("suffix") or match.group("suffix2")

        if not qty or not raw_type:
            continue  # skip malformed matches

        # Normalize type
        norm_key = re.sub(r'[-\s]+', ' ', raw_type.lower()).strip()
        normalized_type = term_map.get(norm_key, norm_key)

        result.append((
            int(qty.strip("()")),
            raw_mod.lower() if raw_mod else None,
            normalized_type,
            raw_suffix.lower() if raw_suffix else None
        ))

    return result


In [69]:
filtered3['match_results'] = filtered3['A_DESCRIPT'].apply(extract_units)
# filtered3['match_results'] = filtered3['A_DESCRIPT'].apply(extract_units2)
filtered3 = filtered3.to_crs('EPSG:4326')

c:\Users\kevan\anaconda3\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [70]:
# create a column for each relevant housing type
housing_types = ['sf_detached', 'sf_attached', 'duplex/triplex', 'multifamily', 'condo']
for h_type in housing_types: 
    filtered3[h_type] = 0

housing_type_dict = {
        'townhouse': 'sf_attached',
        'home': 'sf_detached', 'single family': 'sf_detached', 
        'duplex': 'duplex/triplex', 
        'apartment': 'multifamily', 'multifamily': 'multifamily', 
        'condo': 'condo'}
      
    
# function to fill in housing type columns
def fill_types(match_results):

    row_data = {h_type: 0 for h_type in housing_types}
    for group in match_results:
        
        quantity = int(group[0])
        mod = group[1] if len(group) > 1 else None
        housing = group[2] if len(group) > 2 else None
        
        if housing == 'single family' and mod == 'attached':
            row_data['sf_attached'] += quantity
        elif housing in housing_type_dict:
            row_data[housing_type_dict[housing]] += quantity
        else:
            pass

    return pd.Series(row_data)
            

In [71]:
filtered3[housing_types] = filtered3['match_results'].apply(fill_types)
filtered3.tail(20)

,A_NUMBER,A_TYPE,A_DATE,A_STATUS,A_STATUS_D,A_PROJECT_,A_DESCRIPT,A_USER_ID,A_CASE_PLA,StatCode,...,Creator,EditDate,Editor,geometry,match_results,sf_detached,sf_attached,duplex/triplex,multifamily,condo
20602,D1900073,PL_MINSP,2019-03-21,APP,2022-08-26,Odyssey Townhome Development,165 Townhomes units plus 3 open space lots.,MARCORI,DCULTRA,APP,...,gisproc_sys,2025-05-27,gisproc_sys,POINT (-78.8915 35.92647),"[(165, None, townhouse, None)]",0,165,0,0,0
20713,D2400233,PL_MINSP,2024-10-21,CORR,2025-04-22,Greenleaf Townes,"100 townhomes with public water and sewer, pri...",MICHAELIR,COLERE,CORR,...,gisproc_sys,2025-05-27,gisproc_sys,POINT (-78.80159 35.96787),"[(100, None, townhouse, None)]",0,100,0,0,0
20757,D2400246,PL_MINSP,2024-11-08,UN_RE,2024-11-08,Aura 751,339 multifamily units and associated amenity s...,EARLENETHO,TREYFI,UN_RE,...,gisproc_sys,2025-05-27,gisproc_sys,POINT (-78.9601 35.89581),"[(339, None, multifamily, units)]",0,0,0,339,0
20795,D2400261,PL_MINSP,2024-11-21,UN_RE,2024-11-21,Pointe Grand Durham,Apartment development with a maximum of 260 un...,FRANCISCOM,TREYFI,UN_RE,...,gisproc_sys,2025-05-27,gisproc_sys,POINT (-78.79822 35.97582),"[(260, None, apartment, units)]",0,0,0,260,0
20820,D2400268,PL_MINSP,2024-12-02,UN_RE,2024-12-02,Miami Blvd Multifamily,"4-Multifamily apartment buildings (270 units),...",ELYSSAMO,TREYFI,UN_RE,...,gisproc_sys,2025-05-27,gisproc_sys,POINT (-78.84565 35.93188),"[(4, None, multifamily, None), (270, None, apa...",0,0,0,274,0
20830,D2400270,PL_MINSP,2024-12-03,UN_RE,2024-12-03,Dillard St Apartments (Ph 4BC),"2-4 story, residential buildings with 98 affor...",COLERE,COLERE,UN_RE,...,gisproc_sys,2025-05-27,gisproc_sys,POINT (-78.89406 35.99262),"[(98, None, apartment, units)]",0,0,0,98,0
20847,D2400280,PL_MINSP,2024-12-06,UN_RE,2024-12-06,Croasdaile Village Phase IV,Construction of 21 cottages and 4 multifamily ...,KASIMAZ,JALISAHA,UN_RE,...,gisproc_sys,2025-05-27,gisproc_sys,POINT (-78.93905 36.04664),"[(21, None, multifamily, None)]",0,0,0,21,0
20870,D2400289,PL_MINSP,2024-12-18,UN_RE,2024-12-18,Tribute Rising,Proposed 4 Multifamily apartment buildings (34...,JALISAHA,JALISAHA,UN_RE,...,gisproc_sys,2025-05-27,gisproc_sys,POINT (-78.90518 35.98868),"[(4, None, multifamily, None), (348, None, apa...",0,16,0,352,0
20872,D2400291,PL_MINSP,2024-12-18,UN_RE,2024-12-18,Oakridge Residential Phase C,181 Multi-family units with surface parking.,AKINROTIMIA,PEYTONBL,UN_RE,...,gisproc_sys,2025-05-27,gisproc_sys,POINT (-78.99032 35.95693),"[(181, None, multifamily, units)]",0,0,0,181,0
20941,D2500009,PL_MINSP,2025-01-21,CORR,2025-03-04,Celebrity Rocky Hill Subdivision,Residential subdivision with 42 townhouse lots.,MICHAELMA,COLERE,CORR,...,gisproc_sys,2025-05-27,gisproc_sys,POINT (-78.90479 36.08128),"[(42, None, townhouse, lots)]",0,42,0,0,0


In [72]:
high_school_boundaries = gpd.read_file(r'C:\Users\kevan\OneDrive\Desktop\Data+\regions_2025_26\regions_2025_26.shp') 
high_school_boundaries = high_school_boundaries.to_crs('EPSG:4326')

In [73]:
filtered3_with_school = filtered3.copy()

for i,geometry in enumerate(high_school_boundaries['geometry']):
    
    in_geometry = geometry.contains(filtered3['geometry'])
    high_school_name = high_school_boundaries.loc[i, 'region']

    filtered3_with_school.loc[in_geometry, 'region'] = high_school_name
    

In [74]:
# filtered3_with_school.head(30)

In [75]:
# filtered3_with_school.to_file('filtered3_with_school.json')

In [76]:
# filtered3_with_school.to_file("resdev_cases.geojson", driver="GeoJSON")